In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from ract import Action, RecourseForestClassifier
from datasets import Dataset

In [2]:
DATASETS = ['c', 'f', 'b', 'p']

COST_TYPE = 'MPS'
COST_BUDGETS = [0.1, 0.2, 0.4, 0.5]

N_ESTIMATORS = 200
MAX_DEPTH = 16
ALPHAS = {
    'c': [0.0, 0.02, 0.04, 0.06, 0.08, 0.1], 
    'f': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 
    'b': [0.0, 0.02, 0.04, 0.06, 0.08, 0.1], 
    'p': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
}

KEYS_GROUPBY = [
    'Budget',
    'Method',
    'Alpha',
]
KEYS_MODEL = [
    'Accuracy',
    'AUC',
    'F1',
    'Time [s]',
]
KEYS_ACTION = [
    'Cost',
    'Validity',
    'Budget-Validity',
    'Valid-Cost',
    'Recourse', 
    'Sparsity', 
    'Plausibility',
]

In [3]:
def run_cv_ensemble(dataset='g', causal=False, n_split=10):
    np.random.seed(0)
    
    D = Dataset(dataset=dataset)
    X, y = D.get_dataset()
    params = D.params
    
    results = {
        'Fold': [],
        'Method': [],
        'Alpha': [],
        'Budget': [],
        'Accuracy': [],
        'AUC': [],
        'F1': [],
        'Time [s]': [],
        'Cost': [],
        'Validity': [],
        'Budget-Validity': [],
        'Valid-Cost': [], 
        'Recourse': [], 
        'Sparsity': [],
        'Plausibility': [],
    }
    importance = []
    
    k = 0
    for tr, ts in StratifiedKFold(n_splits=n_split).split(X, y):
        k = k + 1
        X_tr, X_ts, y_tr, y_ts = D.X[tr], D.X[ts], D.y[tr], D.y[ts]

        iforest = IsolationForest(n_estimators=100).fit(X_tr)
        action = Action(X_tr, y_target=0, cost_type=COST_TYPE, causal=causal, **params)
        
        print('- k = {}'.format(k))
        for cost_budget in COST_BUDGETS:
            action.cost_budget = cost_budget

            print('\t- budget: {}'.format(cost_budget))
            for alpha in ALPHAS[dataset]:
                clf = RecourseForestClassifier(action, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH)   
                clf.action.alpha = alpha

                start = time()
                clf = clf.fit(X_tr, y_tr)
                runtime = time() - start
                X_target = X_ts[clf.predict(X_ts) != action.y_target]
                result = clf.explain_action(X_target)
    
                results['Fold'].append(k); results['Method'].append('Baseline' if alpha == 0.0 else 'RACT'); 
                results['Alpha'].append(alpha); results['Budget'].append(cost_budget); 
                results['Accuracy'].append(clf.score(X_ts, y_ts)); results['AUC'].append(roc_auc_score(y_ts, clf.predict_proba(X_ts)[:, 1])); 
                results['F1'].append(f1_score(y_ts, clf.predict(X_ts))); results['Time [s]'].append(runtime); 
                results['Cost'].append(result['cost'].mean()); results['Validity'].append(result['validity'].mean()); 
                results['Budget-Validity'].append(result['cost-validity'].mean()); results['Valid-Cost'].append(result['valid-cost'].mean()); 
                results['Recourse'].append(1 - (~result['cost-validity']).sum() / X_ts.shape[0]); 
                results['Sparsity'].append(result['valid-sparsity'].mean()); 
                results['Plausibility'].append(-1 * iforest.score_samples(result['counterfactual']).mean() if X_target.shape[0]!=0 else 0.0); 
                print('\t\t- Method: {} (alp.: {}) | AUC: {:.4} | Val.: {:.4} | Rec.: {:.4}'.format(results['Method'][-1], alpha, results['AUC'][-1], results['Budget-Validity'][-1], results['Recourse'][-1]))
                importance.append(clf.feature_importances_)

                if alpha == 0.0:
                    clf = RecourseForestClassifier(action, n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, feature_masking=True)

                    start = time()
                    clf = clf.fit(X_tr, y_tr)
                    runtime = time() - start
                    X_target = X_ts[clf.predict(X_ts) != action.y_target]
                    result = clf.explain_action(X_target)

                    results['Fold'].append(k); results['Method'].append('Baseline (OAF)'); 
                    results['Alpha'].append(alpha); results['Budget'].append(cost_budget); 
                    results['Accuracy'].append(clf.score(X_ts, y_ts)); results['AUC'].append(roc_auc_score(y_ts, clf.predict_proba(X_ts)[:, 1])); 
                    results['F1'].append(f1_score(y_ts, clf.predict(X_ts))); results['Time [s]'].append(runtime); 
                    results['Cost'].append(result['cost'].mean()); results['Validity'].append(result['validity'].mean()); 
                    results['Budget-Validity'].append(result['cost-validity'].mean()); results['Valid-Cost'].append(result['valid-cost'].mean()); 
                    results['Recourse'].append(1 - (~result['cost-validity']).sum() / X_ts.shape[0]); 
                    results['Sparsity'].append(result['valid-sparsity'].mean()); 
                    results['Plausibility'].append(-1 * iforest.score_samples(result['counterfactual']).mean() if X_target.shape[0]!=0 else 0.0);                     
                    print('\t\t- Method: {} | AUC: {:.4} | Val.: {:.4} | Rec.: {:.4}'.format(results['Method'][-1], results['AUC'][-1], results['Budget-Validity'][-1], results['Recourse'][-1]))
                    importance.append(clf.feature_importances_)
        
    
    results = pd.DataFrame(results)
    results.to_csv('../res/appendix/budget/{}_forest{}.csv'.format(dataset, '_causal' if causal else ''), index=False)
    
    importance = pd.DataFrame(np.array(importance), columns=D.feature_names)
    importance = pd.concat([results[['Fold', 'Method', 'Alpha', 'Budget']], importance], axis=1)
    importance.to_csv('../res/appendix/budget/fi_{}_forest{}.csv'.format(dataset, '_causal' if causal else ''), index=False)


In [4]:
from datasets import DATASET_FULLNAMES

for dataset in DATASETS: 
    print('# {}'.format(DATASET_FULLNAMES[dataset]))
    run_cv_ensemble(dataset)
    print()

# COMPAS
- k = 1
	- budget: 0.1
		- Method: Baseline (alp.: 0.0) | AUC: 0.7312 | Val.: 0.1202 | Rec.: 0.6677
		- Method: Baseline (OAF) | AUC: 0.7323 | Val.: 0.1984 | Rec.: 0.6661
		- Method: RACT (alp.: 0.02) | AUC: 0.7331 | Val.: 0.3274 | Rec.: 0.7569
		- Method: RACT (alp.: 0.04) | AUC: 0.7267 | Val.: 0.318 | Rec.: 0.7601
		- Method: RACT (alp.: 0.06) | AUC: 0.7249 | Val.: 0.3587 | Rec.: 0.8088
		- Method: RACT (alp.: 0.08) | AUC: 0.7315 | Val.: 0.4551 | Rec.: 0.8428
		- Method: RACT (alp.: 0.1) | AUC: 0.7343 | Val.: 0.5494 | Rec.: 0.8817
	- budget: 0.2
		- Method: Baseline (alp.: 0.0) | AUC: 0.7232 | Val.: 0.3884 | Rec.: 0.7601
		- Method: Baseline (OAF) | AUC: 0.733 | Val.: 0.4259 | Rec.: 0.7488
		- Method: RACT (alp.: 0.02) | AUC: 0.7247 | Val.: 0.3867 | Rec.: 0.7763
		- Method: RACT (alp.: 0.04) | AUC: 0.7319 | Val.: 0.4865 | Rec.: 0.8152
		- Method: RACT (alp.: 0.06) | AUC: 0.7281 | Val.: 0.652 | Rec.: 0.8849
		- Method: RACT (alp.: 0.08) | AUC: 0.7256 | Val.: 0.7019 | Rec.: 0.